In [2]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
from torch.utils.data import DataLoader, Subset

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import kagglehub
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import models
from PIL import Image

/Users/ybr/Developer/ECS-174/ecs174-final-project/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = kagglehub.dataset_download("atulyakumar98/pothole-detection-dataset")

normal_dir = os.path.join(path, 'normal')
potholes_dir = os.path.join(path, 'potholes')
print("Normal images path:", normal_dir)
print("Potholes images path:", potholes_dir)

Normal images path: /Users/ybr/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/normal
Potholes images path: /Users/ybr/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4/potholes


In [4]:
print("Dataset downloaded to:", path)

Dataset downloaded to: /Users/ybr/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4


In [5]:
class PotholeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_names = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(image_path).convert("RGB")

        label = 0 if 'normal' in self.image_dir else 1

        if self.transform:
            image = self.transform(image)


        return image, label

In [6]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


normal_dataset = PotholeDataset(image_dir=normal_dir, transform=transform)
potholes_dataset = PotholeDataset(image_dir=potholes_dir, transform=transform)
dataset = normal_dataset + potholes_dataset

train_indices_file = 'train_indices.npy'
test_indices_file = 'test_indices.npy'

if os.path.exists(train_indices_file) and os.path.exists(test_indices_file):
    # Load the split indices from files
    train_indices = np.load(train_indices_file)
    test_indices = np.load(test_indices_file)
else:
    # Split the dataset into training and test sets
    indices = list(range(len(dataset)))
    random.shuffle(indices)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]

    # Save the split indices to files
    np.save(train_indices_file, train_indices)
    np.save(test_indices_file, test_indices)

# Create subsets using the saved indices
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create data loaders for training, and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
# Criterion
criterion = nn.CrossEntropyLoss()

# MobileNet V2
mobilenet_v2_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet_v2_model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(mobilenet_v2_model.last_channel, 2),
)
mobilenet_v2_optimizer = optim.Adam(mobilenet_v2_model.classifier.parameters(), lr=0.001)

# EfficientNetV2L
efficientnetv2l_model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)
efficientnetv2l_model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(efficientnetv2l_model.classifier[1].in_features, 2),
)
efficientnetv2l_optimizer = optim.Adam(efficientnetv2l_model.classifier.parameters(), lr=0.001)

#ResNet50
resnet50_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, 2)
resnet50_optimizer = optim.Adam(resnet50_model.fc.parameters(), lr=0.001)

#ResNet101
resnet101_model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
resnet101_model.fc = nn.Linear(resnet101_model.fc.in_features, 2)
resnet101_optimizer = optim.Adam(resnet101_model.fc.parameters(), lr=0.001)

#ResNet152
resnet152_model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
resnet152_model.fc = nn.Linear(resnet152_model.fc.in_features, 2)
resnet152_optimizer = optim.Adam(resnet152_model.fc.parameters(), lr=0.001)

# VGG19
vgg19_model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
vgg19_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(vgg19_model.classifier[6].in_features, 2),
)
vgg19_optimizer = optim.Adam(vgg19_model.classifier.parameters(), lr=0.001)


In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
def train_classification_model(name, model, optimizer, train_loader, num_epochs=10):
    model = model.to(device)
    saved_model_path = f"{name}.pth"
    if os.path.exists(saved_model_path):
        print(f"Loading saved model from {saved_model_path}")
        model.load_state_dict(torch.load(saved_model_path, weights_only=True))
        return
    model.train()
    print(f"Training {name}...")
    for epoch in range(num_epochs):
        runningLoss = 0.0
        correctPredictions = 0.0
        totalPredictions = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
            _, predicted = torch.max(outputs, 1)
            totalPredictions += labels.size(0)
            correctPredictions += (predicted == labels).sum().item()

        epochLoss = runningLoss / len(train_loader)
        epochAccuracy = (correctPredictions / totalPredictions) * 100
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epochLoss:.4f}, Accuracy: {epochAccuracy:.2f}%")

    print(f"Finished training {name}")
    torch.save(model.state_dict(), saved_model_path)


In [ ]:
import time
def evaluate_classification_model(model, test_loader):

    start = time.time()

    model.eval()
    correctPredictions = 0.0
    totalPredictions = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            totalPredictions += labels.size(0)
            correctPredictions += (predicted == labels).sum().item()

    accuracy = (correctPredictions / totalPredictions) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    end = time.time()
    print("Time taken:", end - start, "seconds")


In [10]:
train_classification_model("resnet50", resnet50_model, resnet50_optimizer, train_loader)
train_classification_model("resnet101", resnet101_model, resnet101_optimizer, train_loader)
train_classification_model("resnet152", resnet152_model, resnet152_optimizer, train_loader)
train_classification_model("vgg19", vgg19_model, vgg19_optimizer, train_loader)
train_classification_model("mobilenet_v2", mobilenet_v2_model, mobilenet_v2_optimizer, train_loader)
train_classification_model("efficientnet_v2_l", efficientnetv2l_model, efficientnetv2l_optimizer, train_loader)


Loading saved model from resnet50.pth
Loading saved model from resnet101.pth
Loading saved model from resnet152.pth
Loading saved model from vgg19.pth
Loading saved model from mobilenet_v2.pth
Loading saved model from efficientnet_v2_l.pth


In [19]:
evaluate_classification_model(efficientnetv2l_model, test_loader)
evaluate_classification_model(mobilenet_v2_model, test_loader)
evaluate_classification_model(resnet101_model, test_loader)
evaluate_classification_model(resnet50_model, test_loader)
evaluate_classification_model(resnet152_model, test_loader)
evaluate_classification_model(vgg19_model, test_loader)


Test Accuracy: 96.35%
Time taken: 3.08949875831604 seconds
Test Accuracy: 97.81%
Time taken: 1.993407964706421 seconds
Test Accuracy: 97.08%
Time taken: 2.2469348907470703 seconds
Test Accuracy: 99.27%
Time taken: 2.1334340572357178 seconds
Test Accuracy: 99.27%
Time taken: 2.469303846359253 seconds
Test Accuracy: 99.27%
Time taken: 2.4094321727752686 seconds


In [20]:

def evaluate_three_models_at_once(model1, model2, model3, test_loader):
    start = time.time()

    model1.eval()
    model2.eval()
    model3.eval()
    correctPredictions = 0.0
    totalPredictions = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs1 = model1(inputs)
            outputs2 = model2(inputs)
            outputs3 = model3(inputs)
            _, predicted1 = torch.max(outputs1, 1)
            _, predicted2 = torch.max(outputs2, 1)
            _, predicted3 = torch.max(outputs3, 1)
            
            # Majority vote
            for i in range(labels.size(0)):
                votes = [predicted1[i].item(), predicted2[i].item(), predicted3[i].item()]
                final_prediction = max(set(votes), key=votes.count)
                if final_prediction == labels[i].item():
                    correctPredictions += 1
            totalPredictions += labels.size(0)

    accuracy = (correctPredictions / totalPredictions) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    end = time.time()
    print("Time taken:", end - start, "seconds")


import itertools

# List of models
models = [resnet50_model, resnet101_model, resnet152_model, vgg19_model, mobilenet_v2_model, efficientnetv2l_model]
models = {
    'resnet50_model': resnet50_model,
    'resnet101_model': resnet101_model,
    'resnet152_model': resnet152_model,
    'vgg19_model': vgg19_model,
    'mobilenet_v2_model': mobilenet_v2_model,
    'efficientnetv2l_model': efficientnetv2l_model
}

# Go through each possible combination of the models
for model1_name, model2_name, model3_name in itertools.combinations(models.keys(), 3):
    model1 = models[model1_name]
    model2 = models[model2_name]
    model3 = models[model3_name]
    print(f"Evaluating models: {model1_name}, {model2_name}, {model3_name}")
    evaluate_three_models_at_once(model1, model2, model3, test_loader)

Evaluating models: resnet50_model, resnet101_model, resnet152_model
Test Accuracy: 100.00%
Time taken: 3.694265127182007 seconds
Evaluating models: resnet50_model, resnet101_model, vgg19_model
Test Accuracy: 99.27%
Time taken: 3.4504778385162354 seconds
Evaluating models: resnet50_model, resnet101_model, mobilenet_v2_model
Test Accuracy: 99.27%
Time taken: 2.8902969360351562 seconds
Evaluating models: resnet50_model, resnet101_model, efficientnetv2l_model
Test Accuracy: 98.54%
Time taken: 3.9301140308380127 seconds
Evaluating models: resnet50_model, resnet152_model, vgg19_model
Test Accuracy: 100.00%
Time taken: 3.724364995956421 seconds
Evaluating models: resnet50_model, resnet152_model, mobilenet_v2_model
Test Accuracy: 100.00%
Time taken: 3.2510271072387695 seconds
Evaluating models: resnet50_model, resnet152_model, efficientnetv2l_model
Test Accuracy: 100.00%
Time taken: 4.436911106109619 seconds
Evaluating models: resnet50_model, vgg19_model, mobilenet_v2_model
Test Accuracy: 99.2